**A .Introduction:**

***A.1. Description & Discussion of the Background***

An entrepreneur wants to start a new Multi Cuisine Indian restaurant in NEW YORK. Since he believes that New York is one of the most densely populated Indian immigrant states in US. So
He approaches a DataScientist for a suggestion where to start his business. (source from wikipedia)

Since being a Data Scientist we need to solve his problem and recommend the best neighborhood of New York to start his business.

**A.2. Problem :**
1. Finding all the Neighborhoods of New York.
1. Finding all the Neighborhoods with INDIAN Restaurants.
1. Finding all the Neighborhood with-out single INDIAN Restaurants.
1. Finding the best place and most visited INDIAN Restaurant.
1. Suggest best Neighborhood for starting business.

# ***1. Importing Necessary Libraries***

In [32]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')



Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


# ***2. FourSquare Credentials***

In [33]:
CLIENT_ID = 'WLTGOSEFZHTNJDVIPA4GYFQVTZ2TFQWNAPFIM54XAJOYWBLG' # your Foursquare ID
CLIENT_SECRET = 'F32ELTFOMVTL1XKGR4VCGRL3JYL3L4B52WLRTTOS03P4MKGE' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 200
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: WLTGOSEFZHTNJDVIPA4GYFQVTZ2TFQWNAPFIM54XAJOYWBLG
CLIENT_SECRET:F32ELTFOMVTL1XKGR4VCGRL3JYL3L4B52WLRTTOS03P4MKGE


In [34]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

    
    

Data downloaded!


In [46]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)


In [47]:
neighborhoods_data = newyork_data['features']

In [48]:
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

In [87]:
# define the dataframe columns
dfc = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

n1=['Neighborhood','Latitude','Longitude']

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=dfc)

In [88]:
neighborhoods


,Borough,Neighborhood,Latitude,Longitude


In [89]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [90]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [91]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


In [92]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [93]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

#**5. Creating function that retreives all INDIAN resturants in Neighborhoods**

In [94]:
#function that retreives venues using Foursquare 
#Gives all the venues with in given Radius
def get_venues(lat,lng):
    #set variables
    radius=1000
    LIMIT=100

    #url to fetch data from foursquare api
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)
    
    # get all the data
    results = requests.get(url).json()
    venue_data=results["response"]['groups'][0]['items']
    venue_details=[]
    for row in venue_data:
        try:
            venue_id=row['venue']['id']
            venue_name=row['venue']['name']
            venue_category=row['venue']['categories'][0]['name']
            venue_details.append([venue_id,venue_name,venue_category])
        except KeyError:
            pass
        
    column_names=['ID','Name','Category']
    df = pd.DataFrame(venue_details,columns=column_names)
    return df

#***6. Creating function that gets Ratings of all the INDIAN resturants***

In [104]:
#TO find the ratings of the venue

def get_venue_details(venue_id):    
    #url to fetch data from foursquare api
    url = 'https://api.foursquare.com/v2/venues/{}?&client_id={}&client_secret={}&v={}'.format(
            venue_id,
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION)
    
    # get all the data
    results = requests.get(url).json()
    venue_data=results['response']['venue']
    venue_details=[]
    try:
        venue_id=venue_data['id']
        venue_name=venue_data['name']
        venue_likes=venue_data['likes']['count']
        venue_rating=venue_data['rating']
        venue_tips=venue_data['tips']['count']
        venue_details.append([venue_id,venue_name,venue_likes,venue_rating,venue_tips])
    except KeyError:
        pass
        
    column_names=['ID','Name','Likes','Rating','Tips']
    df = pd.DataFrame(venue_details,columns=column_names)
    return df

#***7. Finding all the INDIAN resturants in Neighborhood by calling get_venue function***

In [105]:
# Finding all Neighborhoods having INDIAN resturants

col_names = ['Neighborhood','Count']
# To keep count of how many resturants in each neighboorhood
counts = pd.DataFrame(columns=col_names)

#Actual details of the resturants
column_names=['Neighborhood', 'Id','Name_of_resturant']
indian_resturants=pd.DataFrame(columns=column_names)

for row in zip(neighborhoods['Neighborhood'],neighborhoods['Latitude'], neighborhoods['Longitude']):
  #for each neighborhood
  Neighborhood, Latitude, Longitude = row
  venues = get_venues(Latitude,Longitude)
  #selecting only indian resturants
  indian_resturants_details=venues[venues['Category']=='Indian Restaurant'] 
  #adding no Indian resturants found in each neighborhood
  counts = counts.append({'Neighborhood':Neighborhood,'Count':len(indian_resturants_details)},ignore_index=True)

  for resturant_details in indian_resturants_details.values.tolist():
      id , name , category= resturant_details
      indian_resturants = indian_resturants.append({'Neighborhood': Neighborhood, 
                                                      'Id': id,
                                                      'Name_of_resturant' : name}, ignore_index=True)


KeyError: 'groups'

In [114]:
#finding Number of Restaurants in Each Neighborhood
counts

,Neighborhood,Count
0,Wakefield,0
1,Co-op City,0
2,Eastchester,0
3,Fieldston,0
4,Riverdale,0
5,Kingsbridge,0
6,Marble Hill,0
7,Woodlawn,1
8,Norwood,0
9,Williamsbridge,0


In [115]:
#saving the file instead of computing multiple times
counts.to_csv('Counts.csv')

In [116]:
# List of INDIAN Resturants and their Location
indian_resturants

,Neighborhood,Id,Name_of_resturant
0,Woodlawn,4c0448d9310fc9b6bf1dc761,Curry Spot
1,Parkchester,4c194631838020a13e78e561,Melanies Roti Bar And Grill
2,Spuyten Duyvil,4c04544df423a593ac83d116,Cumin Indian Cuisine
3,Concourse,551b7f75498e86c00a0ed2e1,Hungry Bird
4,Unionport,4c194631838020a13e78e561,Melanies Roti Bar And Grill
5,Bay Ridge,4b5a4dc8f964a520a2bb28e3,Taj Mahal
6,Bay Ridge,545835a1498e820edc6f636f,Bombay Grill
7,Brighton Beach,4af0d31bf964a5207ddf21e3,Pak Nasheman
8,Flatbush,52213c4211d295d4c57a607c,Ashoka Grill
9,Flatbush,564d283d498e6e851df79d87,Great Indian Curry


In [117]:
#saving the file 
indian_resturants.to_csv('Indian_Resturants.csv')

In [118]:
indian_resturants.groupby('Neighborhood')['Id'].count()

Neighborhood
Bay Ridge                    2
Boerum Hill                  1
Brighton Beach               1
Clinton Hill                 2
Concourse                    1
East Flatbush                1
Flatbush                     2
Fort Greene                  1
Fort Hamilton                1
Kensington                   2
Ocean Hill                   2
Parkchester                  1
Prospect Heights             1
Prospect Lefferts Gardens    3
Prospect Park South          2
Spuyten Duyvil               1
Unionport                    1
Washington Heights           1
Woodlawn                     1
Name: Id, dtype: int64

# ***It is evident that Prospect Lefferts Gardens is the Best Location where we find Larger Number of Indian Resturants***

In [119]:
# Resturants in West Ridge
plg = indian_resturants[indian_resturants['Neighborhood']=='Prospect Lefferts Gardens']
plg

,Neighborhood,Id,Name_of_resturant
18,Prospect Lefferts Gardens,4bb93b70cf2fc9b6fe64a002,Gandhi Fine Indian Cuisine
19,Prospect Lefferts Gardens,4afdf78bf964a520862c22e3,King of Tandoor
20,Prospect Lefferts Gardens,4ec80587d3e3e131f2e72835,Bayleaf


In [120]:
#saving all the resturants details in west ridge
plg.to_csv('Resturants_in_Prospect_Lefferts_Gardens.csv')

In [122]:
#getting Ratings of all the resturants in the PROSPECT LEFFERTS GARDENS
column_names=['Neighborhood', 'ID','Name','Likes','Rating']
Prospect_Lefferts_Gardens_resturants_Ratings=pd.DataFrame(columns=column_names)

for row in plg.values.tolist():
    Neighborhood,Id,Name_of_Resturant= row
    try:
        venue_details=get_venue_details(Id)
        id,name,likes,rating,tips=venue_details.values.tolist()[0]
    except IndexError:
        print('No data available for id=',ID)
        # we will assign 0 value for these resturants as they may have been 
        #recently opened or details does not exist in FourSquare Database
        id,name,likes,rating,tips=[0]*5
    Prospect_Lefferts_Gardens_resturants_Ratings = Prospect_Lefferts_Gardens_resturants_Ratings.append({
                                                'Neighborhood': Neighborhood, 
                                                'ID': id,
                                                'Name' : name,
                                                'Likes' : likes,
                                                'Rating' : rating,
                                                'Tips' : tips
                                               }, ignore_index=True)

In [123]:
#all the Ratings of the Resturants in Prospect Lefferts Gardens
Prospect_Lefferts_Gardens_resturants_Ratings.sort_values(by='Rating',ascending=False)

,Neighborhood,ID,Name,Likes,Rating,Tips
0,Prospect Lefferts Gardens,4bb93b70cf2fc9b6fe64a002,Gandhi Fine Indian Cuisine,82,8.6,47.0
1,Prospect Lefferts Gardens,4afdf78bf964a520862c22e3,King of Tandoor,25,7.3,23.0
2,Prospect Lefferts Gardens,4ec80587d3e3e131f2e72835,Bayleaf,6,7.3,7.0


# ***It is evident that Gandhi Fine Indian Cuisine Resturant in Prospect Lefferts Gardens is the best place to dine as it having a rating of 8.6 and 47 likes***

In [124]:
#saving the ratings of each resturants
Prospect_Lefferts_Gardens_resturants_Ratings.to_csv('Ratings.csv')

In [125]:
#finding neighborhood areas with no resturants
Neighborhood_with_no_resturants = counts[counts['Count']==0].reset_index(drop=True)

In [126]:
#finding neighborhoof area with resturants
Neighborhood_areas_with_Resturants = counts[counts['Count']>0].reset_index(drop=True)

In [127]:
Neighborhood_areas_with_Resturants.sort_values(by='Count',ascending=False)

,Neighborhood,Count
14,Prospect Lefferts Gardens,3
9,Kensington,2
16,Prospect Park South,2
15,Ocean Hill,2
5,Bay Ridge,2
7,Flatbush,2
12,Clinton Hill,2
11,Fort Greene,1
17,Fort Hamilton,1
13,Boerum Hill,1


In [136]:
#List of all Indian Resturants in NEW YORK CITY
Indian_Resturants_in_NEWYORK = Neighborhood_areas_with_Resturants.merge(dfc,how='inner',left_on='Neighborhood',right_on='Name') 

TypeError: Can only merge Series or DataFrame objects, a <class 'list'> was passed

In [130]:
Indian_Resturants_in_NEWYORK.drop_duplicates(inplace=True)

NameError: name 'Indian_Resturants_in_NEWYORK' is not defined